In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:
import tensorflow_datasets as tfds 

# Data

In [18]:
mnist_dataset,mnist_info = tfds.load(name='mnist',as_supervised=True,with_info=True)


In [40]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='/home/parth/tensorflow_datasets/mnist/3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [20]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']  # mnist already have test and train datasets made

num_validation = 0.1 * mnist_info.splits['train'].num_examples  #Only Contains the count of samples to be present in validation (No actual data is present)
num_validation = tf.cast(num_validation,tf.int64) # converts a list of item into given data type

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

# Scaling the Features between 0 and 1's

In [41]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image/=255.
    return image ,label
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

# Shuffling The Data to achieve randomness and reduce biasness

In [42]:
BUFFER_size = 10000 # 10000 epoches will be shuffled a once
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_size)

# Extracting Data

In [43]:
#Extract Validation Data
validation_data = shuffled_train_and_validation_data.take(num_validation)
#Extract Train data
train_data = shuffled_train_and_validation_data.skip(num_validation)

# Dividing Data into mini Batches to apply Stochastic Gradient Descent(Batch size= No.of samples = GD and Batch Size=1 = SGD )


In [44]:
Batch_size=100

train_data=train_data.batch(Batch_size)  # Batching is only used in training data not in validation or test data as we dont need to backpropogate in two of them
validation_data = validation_data.batch(num_validation)  # But model excepts only batches so we will convert validation set into one batch
test_data = scaled_test_data.batch(num_test_samples)

validation_input,validation_output = next(iter(validation_data))


# Creating a Network or Model

## Declaring input , output , width(number of neurons in each hidden layer) of model

In [45]:
input_size= 784
output_size =10
width = 50


## Outlining Model

In [46]:
model =tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)), # Defining Input Layer
                            tf.keras.layers.Dense(width,activation='relu'),  # First hidden Layer
                            tf.keras.layers.Dense(width,activation= 'relu'), #Second Hidden Layer
                            tf.keras.layers.Dense(output_size,activation='softmax')  #output layer (as output must be transformed into probabilities we use softmax)
                           ])

## Specify optimizer 

In [47]:
model.compile(optimizer = "adam" , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy']) 

# Training

In [48]:
num_epoch=5
model.fit(train_data,epochs = num_epoch , validation_data = (validation_input,validation_output),verbose=2)

Epoch 1/5
540/540 - 7s - loss: 0.4285 - accuracy: 0.8786 - val_loss: 0.2189 - val_accuracy: 0.9400
Epoch 2/5
540/540 - 3s - loss: 0.1906 - accuracy: 0.9439 - val_loss: 0.1591 - val_accuracy: 0.9548
Epoch 3/5
540/540 - 3s - loss: 0.1449 - accuracy: 0.9570 - val_loss: 0.1280 - val_accuracy: 0.9627
Epoch 4/5
540/540 - 3s - loss: 0.1180 - accuracy: 0.9650 - val_loss: 0.1089 - val_accuracy: 0.9682
Epoch 5/5
540/540 - 3s - loss: 0.1012 - accuracy: 0.9691 - val_loss: 0.1024 - val_accuracy: 0.9707


# Testing

In [49]:
test_loss,test_accuracy = model.evaluate(test_data)
print(test_loss,test_accuracy)

1/1 [==============================] - 1s 981ms/step - loss: 0.1193 - accuracy: 0.9624
0.11926550418138504 0.9624000191688538
